In [ ]:
from taichi_em_discovery import EMDiscovery
import numpy as np

B, N = 100_000, 64
dt, T = 0.1, 1000

sim = EMDiscovery(B=B, N=N, dt=dt, T=T, max_abs_a=8.0, beta_ema=0.1, seed=1)
sim.run()

K = sim.K_numpy()
activity, ema, flips, overflow, steps = sim.metrics_numpy()
idx, score = EMDiscovery.select_interesting(
    K, activity, ema, flips, overflow,
    eps_converge=2e-4, lambda_flip=0.05, top_k=1000
)

A = sim.state_numpy()  # optional
K_sel = K[idx]
A_sel = A[idx]
print("Selected:", len(idx), "Top score:", score[idx[0]])
print("n_overflow:", overflow.sum(), "/", B)


In [ ]:
K_sel[0:10,:]

In [ ]:
from em_numpy_single import simulate_single_em
import matplotlib.pyplot as plt

K_test=K_sel[3,:] #change the index to run different theta 
print("k_test",K_test)

# Pick one of the top-K params from the Taichi discovery pass
K_star = K_test.astype(np.float32)  # example

a_final, overflow, traj = simulate_single_em(
    K_star,
    N=64,
    dt=0.1,
    T=200,
    max_abs_a=5.0,
    seed=0,
    A0=None,           # or pass a custom initial state (N,)
    return_traj=True,  # set False if you only need the last state
)

print("overflow:", overflow, "final shape:", a_final.shape)

# if traj is not None: plot or inspect it: shape (T+1, N)
plt.figure(figsize=(15, 15))
plt.imshow(traj)
plt.colorbar()
plt.show()